In [ ]:
!pip uninstall -y transformers
!pip install git+https://github.com/huggingface/transformers
!pip install autoawq

!pip install yfinance
!pip install gnews
!pip install jugaad-data

!pip install gradio

In [ ]:
from transformers import AutoModelForCausalLM, AutoModelForSequenceClassification, AutoTokenizer, TextStreamer
import torch

import numpy as np
import matplotlib.pyplot as plt

from data_api import StockInfo

import gradio as gr
import pandas as pd

/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
si = StockInfo()

print(device)

[*********************100%%**********************]  1 of 1 completed

cuda:0


In [ ]:
model = AutoModelForCausalLM.from_pretrained("TheBloke/finance-chat-AWQ",
                                             torch_dtype='auto',
                                             trust_remote_code=True).to(device)

tokenizer = AutoTokenizer.from_pretrained("TheBloke/finance-chat-AWQ", trust_remote_code=True)

sentiment_model = AutoModelForSequenceClassification.from_pretrained('RashidNLP/Finance-Sentiment-Classification',
                                                                     num_labels = 3).to(device)

sentiment_tokenizer = AutoTokenizer.from_pretrained("RashidNLP/Finance-Sentiment-Classification", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/916 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:147: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be prevail.
  warnings.warn(warning_msg)
You have loaded an AWQ model on CPU and have a CUDA device available, make sure to set your model on a GPU device in order to run your model.
We suggest you to set `torch_dtype=torch.float16` for better efficiency with AWQ.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/3.89G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/193 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/413 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.65M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [ ]:
def prompt_genrator(ticker,stock_data,benchmark_data):

    stock_name = ticker.split('.')[0]

    __questions__ = [f'Give a short summary on {stock_name} stock',
                     f'Is {stock_name} a good stock?',
                     f'Tell me about {stock_name} stock performace',
                     f'brief {stock_name} performance']

    prompt = 'Use these fact to answer the question:\n\n\n'
    prompt += f'{stock_name} stock performance and metrics\n\n'

    for i in stock_data.keys():
        if stock_data[i] != 0 and stock_data[i] != 'NIL':
            prompt += (i + ' : ')
            prompt += (str(stock_data[i])+'\n')

    prompt += '\n\nBenchmark assets performance\n\n'

    for i in benchmark_data.keys():
        if benchmark_data[i] != 0 and benchmark_data[i] != 'NIL':
            prompt += (i + ' : ')
            prompt += (str(benchmark_data[i])+'\n')

    prompt += '\n\n'+np.random.choice(__questions__)

    return prompt

def pipeline(task,ticker,history):
    ticker = ticker + '.NS'

    # Retrive Information
    if task == 'StockSummary':
        history = ""
        fig,sd,bd = si.StockPerformanceMetrics(ticker)

        # Create and tokenize prompt
        prompt = prompt_genrator(ticker,sd,bd)

        system_prompt = "\nYou are a helpful, respectful and honest financial assistant. If you don't know the answer to a question, please don't share false information.\n" # Please do NOT change this
        final_prompt = f"<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n{prompt} [/INST]"

        input = tokenizer(final_prompt,return_tensors='pt').input_ids.to(device)

        # Genration hyperparameters and Prompt completion
        generation_params = {
            "do_sample": True,
            "temperature": 0.6,
            "top_p": 0.95,
            "top_k": 40,
            "max_new_tokens": 512,
            "repetition_penalty": 1.1}
        output = model.generate(input,**generation_params)
        result = tokenizer.decode(output[0][len(input[0]):])
        for i in result.split():
            history += (i+" ")
            yield fig,history

    if task == 'StockSentiment':

        news = si.fetch_recent_devlopments(ticker)
        complete_context = history + '\n' + news

        input = sentiment_tokenizer(complete_context,max_length = 512,return_tensors='pt').input_ids.to(device)

        logits_output = sentiment_model(input).logits
        probability_output = torch.nn.functional.softmax(logits_output, dim = 1)[0].cpu().detach().numpy()
        labels = ['Negative','Neutral','Positive']
        colors = ['#ff371c','#1c96ed','#49ff42']
        explode = [0,0,0]
        explode[np.argmax(probability_output)] = 0.1

        fig = plt.figure()
        sub = fig.add_subplot(111)

        sub.pie(probability_output,labels=labels,colors=colors,explode=explode,shadow=True)
        sub.set_title(f'Sentiments toward {ticker.split(".")[0]} stock')

        yield fig,news

## Build Gradio Based Graphical User interface

In [ ]:
availabel_tickers = pd.read_csv('EQUITY_L.csv')['SYMBOL'].to_list()

with gr.Blocks() as app:

    with gr.Row():
        with gr.Column():
            ticker_ = gr.Dropdown(availabel_tickers,label='Choose Ticker',info='Choose any NSE listed Stock ticker')
            action_buttn = gr.Button("RUN")

    with gr.Row():
        with gr.Column():
            plot = gr.Plot(label='Past 1 year price movement')
        with gr.Column():
            text = gr.TextArea(label='Stock Analysis',info='Summary of stock performance and other key metrics.')

    with gr.Row():
        news_buttn = gr.Button("GET HEADLINES")

    with gr.Row():
        with gr.Column():
            news_txt = gr.TextArea(label='Recent Devlopments and Headlines')
        with gr.Column():
            sentiment_plot = gr.Plot()

    action_buttn.click(pipeline,inputs=[gr.Text('StockSummary',visible=False),ticker_,text],outputs=[plot,text])
    news_buttn.click(pipeline,inputs=[gr.Text('StockSentiment',visible=False),ticker_,text],outputs=[sentiment_plot,news_txt])

app.launch(share=True,debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://f6c80ae589eb7a2d6e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
